## Reference Article
Torrefied biomass quality prediction and optimization using machine learning algorithms https://doi.org/10.1016/j.ceja.2024.100620

# 1. Feature Selection
Particle Swarm Optimization (PSO) and Generic Algorithms (GA) is used for feature selection and hyperparameter tuning.

# 2. DATA
## 2.1. Raw Data w/o added columns and all fuels included
No feature engineering, just raw columns

sample,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,h,o,n,s,cl,hc,oc,lhv,devol_yield,x0_biomass,x0_coal,x0_mix,x0_plastic



In [81]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer


# Load dataset
data_path =  r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\Data\separate_samples\encoded_shuffled.csv"
data = pd.read_csv(data_path)

# Drop unnecessary columns
data = data.drop(columns=['x0_biomass', 'x0_plastic', 'x0_mix', 'x0_coal', 'sample'])

# Split before imputation & scaling (avoiding data leakage)
X = data.drop(columns=['devol_yield'])  # Features
y = data['devol_yield']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply KNN Imputer
imputer = KNNImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Convert back to DataFrame
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X.columns)

# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Convert back to DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)


## 2.2. Biomass Only Data

In [141]:

# Load dataset
data_path =  r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\Data\separate_samples\encoded_shuffled.csv"
data = pd.read_csv(data_path)
print('Overall Data Shape:', data.shape)

biomass_data = data.loc[(data['x0_biomass'] == 1)]
plastic_data = data.loc[(data['x0_plastic'] == 1)]
coal_data = data.loc[(data['x0_coal'] == 1)]

data = pd.concat([biomass_data, plastic_data, coal_data])
data = data.drop(columns=['x0_biomass', 'x0_plastic', 'x0_mix', 'x0_coal', 'sample'])
print('Biomass Only Data Shape:', data.shape)
# Drop unnecessary columns


# Split before imputation & scaling (avoiding data leakage)
X = data.drop(columns=['devol_yield'])  # Features
y = data['devol_yield']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply KNN Imputer
imputer = KNNImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Convert back to DataFrame
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X.columns)

# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Convert back to DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)


Overall Data Shape: (1990, 23)
Biomass Only Data Shape: (1427, 18)


## 3. Simple Baselines
Mean Predictor: Always predicts the mean of the training targets.

Median Predictor: Always predicts the median.

k-Nearest Neighbors Regression (KNN): Non-parametric method using neighbors' average.

In [124]:
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor

# Dummy models
mean_model = DummyRegressor(strategy="mean")
median_model = DummyRegressor(strategy="median")

# KNN Regression
knn_model = KNeighborsRegressor(n_neighbors=5)


In [125]:
mean_model.fit(X_train_scaled, y_train)
median_model.fit(X_train_scaled, y_train)
knn_model.fit(X_train_scaled, y_train)

KNeighborsRegressor()

In [126]:
mean_predictions = mean_model.predict(X_test_scaled)
median_predictions = median_model.predict(X_test_scaled)
knn_predictions = knn_model.predict(X_test_scaled)


In [127]:
print('Mean Model Score:', mean_model.score(X_test_scaled, y_test))
print('Median Model Score:',median_model.score(X_test_scaled, y_test))
print('KNN Model Score:', knn_model.score(X_test_scaled, y_test))

Mean Model Score: -0.0009893102761686645
Median Model Score: -0.001573357722227664
KNN Model Score: 0.6962937348136864


In [128]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("Mean Model MAE:", mean_absolute_error(y_test, mean_predictions))
print("Median Model MAE:", mean_absolute_error(y_test, median_predictions))
print("KNN Model MAE:", mean_absolute_error(y_test, knn_predictions))

Mean Model MAE: 18.423335507498155
Median Model MAE: 18.42619263996939
KNN Model MAE: 8.808247501925509


## 2. Linear Models
Linear Regression (OLS)

Ridge Regression (L2 regularization)

Lasso Regression (L1 regularization)

Elastic Net (Combination of L1 and L2)


In [129]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

lin_model = LinearRegression()
ridge_model = Ridge(alpha=1.0)
lasso_model = Lasso(alpha=0.1)
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5)

In [130]:
lin_model.fit(X_train_scaled, y_train)
ridge_model.fit(X_train_scaled, y_train)
lasso_model.fit(X_train_scaled, y_train)
elastic_model.fit(X_train_scaled, y_train)

ElasticNet(alpha=0.1)

In [131]:
print('Linear Model Score:',lin_model.score(X_test_scaled, y_test))
print('Ridge Model Score:',ridge_model.score(X_test_scaled, y_test))
print('Lasso Model Score:',lasso_model.score(X_test_scaled, y_test))
print('Elastic Model Score:',elastic_model.score(X_test_scaled, y_test))

Linear Model Score: 0.49017455625117423
Ridge Model Score: 0.49139357980348586
Lasso Model Score: 0.5014989781718981
Elastic Model Score: 0.49946967014457855


## 3. Tree-Based Models
Decision Tree Regression: Simple non-linear model.

Random Forest Regression: Ensemble of decision trees.

Gradient Boosting Regression (GBR)

XGBoost

LightGBM
CatBoost

In [132]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

dt_model = DecisionTreeRegressor(max_depth=5)
rf_model = RandomForestRegressor(n_estimators=100, max_depth=5)
gbr_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)

xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1)
lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1)

In [133]:
dt_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)
gbr_model.fit(X_train_scaled, y_train)
xgb_model.fit(X_train_scaled, y_train)
lgb_model.fit(X_train_scaled, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000058 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 133
[LightGBM] [Info] Number of data points in the train set: 780, number of used features: 17
[LightGBM] [Info] Start training from score 54.387338
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

LGBMRegressor()

In [134]:
dt_predictions = dt_model.predict(X_test_scaled)
gbr_predictions = gbr_model.predict(X_test_scaled)
lgb_predictions = lgb_model.predict(X_test_scaled)
xgb_predictions = xgb_model.predict(X_test_scaled)
rf_predictions = rf_model.predict(X_test_scaled)

In [135]:
print('Decision Tree Regressor Score:', mean_squared_error(y_test, dt_predictions))
print('Gradient Boosting Regressor Score:', mean_squared_error(y_test, gbr_predictions))
print('lightGBM Regressor Score:', mean_squared_error(y_test, lgb_predictions))
print('xgboost Regressor Score:', mean_squared_error(y_test, xgb_predictions))
print('RF Regressor Score:', mean_squared_error(y_test, rf_predictions))

Decision Tree Regressor Score: 176.9808149211249
Gradient Boosting Regressor Score: 143.82443029109263
lightGBM Regressor Score: 144.87778862083883
xgboost Regressor Score: 137.66905243925845
RF Regressor Score: 166.91134623822475


In [136]:
print('Decision Tree Regressor Score:', dt_model.score(X_test_scaled, y_test))
print('Gradient Boosting Regressor Score:', gbr_model.score(X_test_scaled, y_test))
print('lightGBM Regressor Score:', lgb_model.score(X_test_scaled, y_test))
print('XGBoost Regressor Score:', xgb_model.score(X_test_scaled, y_test))
print('RF Regressor Score:', rf_model.score(X_test_scaled, y_test))

Decision Tree Regressor Score: 0.6788274659582245
Gradient Boosting Regressor Score: 0.738997378024892
lightGBM Regressor Score: 0.7370858162312062
XGBoost Regressor Score: 0.7501677317354863
RF Regressor Score: 0.6971008408140396


## 4. Nonlinear & Probabilistic Models
Gaussian Process Regression (GPR)

Support Vector Regression (SVR)

Neural Networks (MLP)

In [137]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

gpr_model = GaussianProcessRegressor()
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
mlp_model = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', max_iter=2000)

In [138]:
gpr_model.fit(X_train_scaled, y_train)
svr_model.fit(X_train_scaled, y_train)
mlp_model.fit(X_train_scaled, y_train)

C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(max_iter=2000)

In [139]:
print('Gaussian Process Regressor Score:', gpr_model.score(X_test_scaled, y_test))
print('SVR Score:', svr_model.score(X_test_scaled, y_test))
print('MLP Score:', mlp_model.score(X_test_scaled, y_test))

Gaussian Process Regressor Score: 0.4865836492917507
SVR Score: 0.4337891492275485
MLP Score: 0.7134692451551707


## Evaluation of the Models

In [140]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

models = {
    "Linear Regression": lin_model,
    "Ridge": ridge_model,
    "Lasso": lasso_model,
    "Elastic Net": elastic_model,
    "Decision Tree": dt_model,
    "Random Forest": rf_model,
    "Gradient Boosting": gbr_model,
    "XGBoost": xgb_model,
    "LightGBM": lgb_model,
    "GPR": gpr_model,
    "SVR": svr_model,
    "MLP": mlp_model,
}

results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    results[name] = {
        "R2": r2_score(y_test, y_pred),
        "RMSE": mean_squared_error(y_test, y_pred),
        "MAE": mean_absolute_error(y_test, y_pred),
    }

for model, metrics in results.items():
    print(f"{model}: R2 = {metrics['R2']:.4f}, RMSE = {metrics['RMSE']:.4f}, MAE = {metrics['MAE']:.4f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000063 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 133
[LightGBM] [Info] Number of data points in the train set: 780, number of used features: 17
[LightGBM] [Info] Start training from score 54.387338
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

C:\Users\demir\PycharmProjects\volatile-predict\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


# Results
All data, train-test-split 